In [3]:
#✅ Cell 1: Import Dependencies
# 📦 Import Dependencies
import os
import sys
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
import ipywidgets as widgets
import pandas as pd

# Ensure project root is in sys.path for local imports
PROJECT_ROOT = os.path.dirname(os.path.dirname(os.path.abspath("__file__")))
sys.path.append(PROJECT_ROOT)

from models.autoencoder import ConvAutoencoder


In [4]:
#⚙️ Cell 2: Config + Load Model
# ⚙️ Config
IMAGE_SIZE = (128, 128)
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_PATH = os.path.join(PROJECT_ROOT, "models", "checkpoints", "autoencoder.pth")
THRESHOLD_PATH = os.path.join(PROJECT_ROOT, "config", "settings.yaml")

import yaml
with open(THRESHOLD_PATH, "r") as f:
    settings = yaml.safe_load(f)
threshold = settings.get("anomaly_threshold", 0.015)

# Load Model
def load_model():
    model = ConvAutoencoder().to(DEVICE)
    model.load_state_dict(torch.load(MODEL_PATH, map_location=DEVICE))
    model.eval()
    return model

model = load_model()


In [5]:
#🔁 Cell 3: Inference Functions
# 📐 Define transforms
transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.ToTensor()
])

# 🔁 Reconstruction + scoring
def infer_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image_tensor = transform(image).unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        reconstruction = model(image_tensor)
        loss = F.mse_loss(reconstruction, image_tensor).item()
    return image, reconstruction.squeeze().cpu(), round(loss, 5)


In [6]:
#🖼️ Cell 4: Visual Comparison
# 🖼️ Visualize input vs reconstruction
def show_comparison(image_path, threshold=threshold):
    original, reconstructed, score = infer_image(image_path)

    fig, axes = plt.subplots(1, 2, figsize=(10, 5))
    axes[0].imshow(original)
    axes[0].set_title("Original")
    axes[0].axis("off")

    axes[1].imshow(reconstructed.permute(1, 2, 0))
    axes[1].set_title("Reconstruction")
    axes[1].axis("off")

    verdict = "✅ REAL" if score < threshold else "⚠️ SUSPECT / FAKE"
    color = "green" if verdict == "✅ REAL" else "red"
    plt.suptitle(f"Score: {score}  →  {verdict}", fontsize=14, color=color)
    plt.show()


In [7]:
#🔍 Cell 5: Image Selector Dropdown
# 🔍 File selector dropdown
SAMPLE_DIR = os.path.join(PROJECT_ROOT, "data", "preprocessed", "fertilizer", "YaraMila")
image_files = [f for f in os.listdir(SAMPLE_DIR) if f.endswith((".jpg", ".jpeg", ".png"))]
dropdown = widgets.Dropdown(options=image_files, description="Select Image:")

def on_select(change):
    sample_path = os.path.join(SAMPLE_DIR, change["new"])
    show_comparison(sample_path)

dropdown.observe(on_select, names="value")
display(dropdown)


Dropdown(description='Select Image:', options=('000001.jpg', '000002.jpg', '000003.jpg', '000004.jpg', '000005…

In [13]:
#📊 Cell 6: Batch Scoring + Save CSV
# 🧪 Score all images in a directory and log anomalies
def batch_score_and_log(folder, threshold=threshold):
    results = []
    for file in sorted(os.listdir(folder)):
        if file.lower().endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(folder, file)
            _, _, score = infer_image(image_path)
            verdict = "REAL" if score < threshold else "SUSPECT / FAKE"
            results.append({"file": file, "score": score, "result": verdict})

    df = pd.DataFrame(results)
    df = df.sort_values("score", ascending=False).reset_index(drop=True)

    print("📈 Top 10 Highest Anomaly Scores")
    display(df.head(10))

    output_csv = os.path.join(PROJECT_ROOT, "data", "annotations", "batch_scores.csv")
    df.to_csv(output_csv, index=False)
    print(f"✅ Results saved to: {output_csv}")

# Run:
batch_score_and_log(SAMPLE_DIR)


📈 Top 10 Highest Anomaly Scores


,file,score,result
0,000028.jpg,0.01259,REAL
1,000025.jpg,0.01219,REAL
2,000019.jpg,0.01143,REAL
3,000015.jpg,0.01126,REAL
4,000017.jpg,0.01007,REAL
5,000003.jpg,0.00787,REAL
6,000014.jpg,0.00786,REAL
7,000004.jpg,0.00766,REAL
8,000026.jpg,0.00750,REAL
9,000029.jpg,0.00747,REAL


✅ Results saved to: d:\Documents\CODE_WITH_ERICADESHH\GitHub\HarvestGuard\data\annotations\batch_scores.csv
